# ResNet50模型训练ImageNet子集
   
### 实验内容
    通过ResNet50模型及其预训练参数提取图像特征，进行图像分类。
    
### 实验目的
    观察ResNet50模型及预训练参数对图像分类的性能表现。
    
### 实验步骤
    1.加载 tf.keras.applications.ResNet50 及其预训练参数
    2.追加3层Dense输出10分类的模型
    3.训练模型
    4.测试模型
    
### 实验数据
    抽取ImageNet子集10子分类作为实验数据
    
    
### 实验结果

### 参考资料


In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

data_path='%s/work/data/imagenet/subset'%os.getenv('HOME')          #数据目录
out_path='%s/work/data/labs_out/lab_imagenet-subset-resnet50'%os.getenv('HOME') #输出目录
log_path  ='%s/log_dir'%out_path    #输出日志目录
preds_path='%s/predicts'%out_path   #预测结果
cp_file='%s/cp_file.h5'%out_path    #训练断点
model_file='%s/model.h5'%out_path   #模型文件

input_shape=(224,224,3)
target_size=(224,224)
epochs=3
num_class=10
batch_size=8


Using TensorFlow backend.


# 构造数据生成器

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

#构造图像数据生成器:train
gen_train = ImageDataGenerator(
        featurewise_center           =True,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.2,
        width_shift_range            =0.2,
        height_shift_range           =0.2,
        shear_range                  =0.2,
        zoom_range                   =0.2,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =True,
        vertical_flip                =True,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_train=gen_train.flow_from_directory(directory='%s/train'%(data_path)
                                         ,batch_size=batch_size
                                         ,target_size=target_size)
#构造图像数据生成器:valid
gen_valid = ImageDataGenerator(
        featurewise_center           =False,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.,
        width_shift_range            =0.,
        height_shift_range           =0.,
        shear_range                  =0.,
        zoom_range                   =0.,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =False,
        vertical_flip                =False,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_valid=gen_valid.flow_from_directory(directory='%s/valid'%(data_path)
                                         ,batch_size=batch_size
                                         ,target_size=target_size)

#构造图像数据生成器:test
gen_test = ImageDataGenerator(
        featurewise_center           =False,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.,
        width_shift_range            =0.,
        height_shift_range           =0.,
        shear_range                  =0.,
        zoom_range                   =0.,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =False,
        vertical_flip                =False,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_test=gen_test.flow_from_directory(directory='%s/test'%(data_path)
                                       ,batch_size=batch_size
                                       ,shuffle=False                                       
                                       ,target_size=target_size)

Found 13000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.
Found 969 images belonging to 10 classes.


# 模型构建

In [3]:
from keras import models,layers,optimizers
from keras.applications.resnet50 import ResNet50

#创建基础模型
base_model = ResNet50(weights='imagenet', include_top=False,input_shape=input_shape)

# 添加 flatten layer
x = base_model.output
x = layers.Flatten(name='flatten_1')(x)
# 添加 Dense layer
x = layers.Dense(4096, activation='relu',name='dense_1')(x)
x = layers.Dense(4096, activation='relu',name='dense_2')(x)
predictions = layers.Dense(10, activation='softmax',name='dense_o')(x)

# 构建训练模型
model = models.Model(input=base_model.input, output=predictions)

#打印模型
model.summary()
for i,layer in enumerate(model.layers):
    print('%d:%s-%s'%(i,layer.name,layer.trainable))

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  app.launch_new_instance()


# 模型训练

In [4]:
#断点训练:monitor监控参数可以通过score = model.evaluate(x_test, y_test, verbose=0)的score查询
checkpoint_cb = keras.callbacks.ModelCheckpoint(cp_file, monitor='val_acc', verbose=1, save_best_only=True, mode='auto',period=1)
#EarlyStopping
earlyStopping_cb=keras.callbacks.EarlyStopping(monitor='acc', patience=10, verbose=0, mode='max')
#TensorBoard
tensorBoard_cb=keras.callbacks.TensorBoard(log_dir=log_path)
#回调函数序列
callbacks_list = [checkpoint_cb,earlyStopping_cb,tensorBoard_cb]
#callbacks_list = [checkpoint_cb,tensorBoard_cb]

# 解冻基础模型
for layer in base_model.layers:
    layer.trainable = True
    
#断点加载
if os.path.exists(cp_file):
    print('load check point:%s',cp_file)
    model.load_weights(cp_file,by_name=True)
    
# 冻结基础模型
for layer in base_model.layers:
    layer.trainable = True

# 编译模型
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])
   
# 训练少量数据，以适应新数据集
history = model.fit_generator(
  data_train,
  steps_per_epoch=10, #np.ceil(data_train.samples/batch_size),
  epochs=100,
  validation_data=data_valid,
  validation_steps=50,
  callbacks=callbacks_list)

#保存模型
print('savel model:',model_file)
model.save(model_file)

Epoch 1/100


/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


ResourceExhaustedError: OOM when allocating tensor with shape[100352,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/RMSprop/mul_638 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](RMSprop/lr/read, training/RMSprop/gradients/dense_1/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
if False:
    # 重新设置可训练的层
    train_start=91
    for layer in model.layers[:train_start]:
       layer.trainable = False
    for layer in model.layers[train_start:]:
       layer.trainable = True

    #重新编译模型，并且调低学习率
    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['acc'])

    #断点加载
    if os.path.exists(cp_file):
        model.load_weights(cp_file,by_name=True)

    #模型训练
    history = model.fit_generator(
      data_train,
      steps_per_epoch=np.ceil(data_train.samples/batch_size),
      epochs=epochs,
      validation_data=data_valid,
      validation_steps=50,
      callbacks=callbacks_list)
    print('history:',history.history)

    #保存模型
    model.save(model_file)

# 模型测试

In [ ]:
from mylibs.predicts_to_symlink import predicts_to_symlink
#计算精度
def compute_acc(y_pred,y_true):
    acc=(y_pred-y_true)==0
    return acc.sum()/acc.size

#加载模型
model.load_weights(model_file)

#模型测试
print('predicting beginning ......')
#type(y_pred)=> <class 'numpy.ndarray'>
y_pred=model.predict_generator(
    data_test, 
    steps=None, #预测轮数
    max_queue_size=32, 
    workers=1, 
    use_multiprocessing=False, 
    verbose=1)

#输出软链接目录
predicts_to_symlink(y_pred,'%s/test'%data_path,preds_path,data_test)

#准确率计算
acc=compute_acc(np.argmax(y_pred,axis=1),data_test.classes)
print('samples:',data_test.samples)
print('classes[:2]:')
print(data_test.classes[:2])
print('y_pred.shape:',y_pred.shape)
print('y_pred[:2]:')
print(y_pred[:2])
print('准确率:',acc)